#### 2018/11/1<br>V1.0_0.6231<br>因为在计算ctr的时候是使用了data.groupby().agg()的，所以过拟合现象严重！

#### <font color=red>2018/11/2<br>V1.2_0.7128<br>1.建立在V1.0版本上；<br>2.将data.groupby.agg改成了traindf.grouby.agg，因为data中有一部分的label是-1，所以对成绩产生了非常大的噪声；<br>3.将点击与否的界限从0.5改成了0.4；<br>4.将原先使用lgb.train改成了LGBMClassifier.fit，但是对成绩本身的影响应该不是特别大；<font>  

#### <font color=green>2018/11/3<br>V1.3<br>1.建立在V1.2的基础上；<br>1.对于部分0值变量重新赋予一个新值-1，从而方便使用scaler；<br>2.使用LogisticRegressor进行训练；<font>

#### <font color=blue>2018/11/3<br>V1.4<br>1.对于NaN变量赋予新值-1,重新再跑一遍V1.2模型；<br>2.将V1.3和本次模型进行lgb ensemble；<font>

#### <font color=black>2018/11/3<br>V1.5<br>1.ensemble with lr；<br>2.fillna('-1')然后再LabelEncoder()；<font>

#### <font color=red>2018/11/3<br>V1.6<br>1.combine V1.3 and V1.4；<br>2.将所有函数进行封装，从而达到自动化效果；<font>

# ==========base data solve==========

In [58]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import gc
gc.collect()

314

In [59]:
def base_data(data):
    data['query_len']=data['query'].apply(lambda x:len(x.replace('{','').replace('}','').split(',')))
    data['prefix_len']=data['prefix'].apply(lambda x:len(str(x)))
    data['title_len']=data['title'].apply(lambda x:len(str(x))) 
    
    le=LabelEncoder()
    le_cols=['prefix','title','query','tag']
    for feature in le_cols:
        data[feature]=data[feature].astype(str)
    data['prefix']=le.fit_transform(data['prefix'])
    data['title']=le.fit_transform(data['title'])
    data['query']=le.fit_transform(data['query'])
    data['tag']=le.fit_transform(data['tag'])
    
    return data

In [60]:
def get_ctr(data):
    traindf=data[data['label']!=-1]
    testdf=data[data['label']==-1]
    print(traindf.shape)
    print(testdf.shape)
    
    col=['prefix','query','title','tag','prefix_len','query_len','title_len']
    for item in col:
        temp=traindf.groupby(item,as_index=False)['label'].agg({item+'_click':'sum',item+'_count':'count'})
        temp[item+'_ctr']=(temp[item+'_click']/temp[item+'_count'])
        traindf = pd.merge(traindf, temp, on=item, how='left')
        testdf=pd.merge(testdf,temp,on=item,how='left')
    
    for i in range(len(col)):
        for j in range(i+1,len(col)):
            item_g=[col[i],col[j]]
            temp=traindf.groupby(item_g,as_index=False)['label'].agg({'_'.join(item_g)+'_click':'sum','_'.join(item_g)+'_count':'count'})
            temp['_'.join(item_g)+'_ctr']=temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'_count']+3)
            traindf=pd.merge(traindf,temp,on=item_g,how='left')
            testdf=pd.merge(testdf,temp,on=item_g,how='left')
    
    return traindf,testdf

In [61]:
# def getxy(traindf,testdf):
#     testddf=testdf.drop(['label'],axis=1)
#     print(traindf.shape,testddf.shape)

#     y=traindf.pop('label').values
#     X=traindf.as_matrix()
#     testX=testddf.as_matrix()
    
#     return X,y,testX

# =========LogisticRegression=======

In [62]:
def lr_fit(clf,X,y,testX,N):
    ft=0
    fv=0
    sub_train=pd.DataFrame()
    sub_test=pd.DataFrame()
    skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)
    for i,(train_index,test_index) in enumerate(skf.split(X,y)):
        print("Fold=%s"%i)
        clf.fit(X[train_index],y[train_index])
        train_pred=clf.predict(X[train_index])
        trainf=f1_score(y[train_index],train_pred)
        print("f1 in train set:",trainf)
        ft+=trainf
    
        val_pred=clf.predict(X[test_index])
        valf=f1_score(y[test_index],val_pred)
        print("f1 in test set:",valf)
        fv+=valf
    
        train_all=clf.predict_proba(X)[:,1]
        sub_train['ans_%s'%i]=train_all
    
        test_proba=clf.predict_proba(testX)[:,1]
        sub_test['ans_%s'%str(i)]=test_proba
    print("mean f1 in train set:%.5f"%(ft/N))
    print("mean f1 in val set:%.5f"%(fv/N))

    return sub_train,sub_test

# =============lgb============

In [63]:
def lgb_fit(clf,X,y,testX,N):
    ft=0
    fv=0
    
    sub_train=pd.DataFrame()
    sub_test=pd.DataFrame()

    skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)
    for i,(train_index,test_index) in enumerate(skf.split(X,y)):
        print("Fold=%s"%(i))
        model=clf.fit(X[train_index],y[train_index],
                 eval_names=['train','valid'],
                 eval_metric='logloss',
                  eval_set=[(X[train_index],y[train_index]),(X[test_index],y[test_index])],
                 early_stopping_rounds=100,
                  verbose=100,)

        print("best logloss on train_set:",(model.best_score_['train']['binary_logloss']))
        print("best logloss on test_set:",(model.best_score_['valid']['binary_logloss']))
    
        train_proba=model.predict_proba(X[train_index],num_iteration=model.best_iteration_)[:,1]
        ftrain=f1_score(y[train_index],(np.where(train_proba>0.37,1,0)))
        print("f1_score on train_set:%.5f"%ftrain)
        ft+=ftrain
    
        val_proba=model.predict_proba(X[test_index],num_iteration=model.best_iteration_)[:,1]
        fval=f1_score(y[test_index],(np.where(val_proba>0.37,1,0)))
        print("f1_score on test_set:%.5f"%fval)
        fv+=fval
    
        train_all=model.predict_proba(X,num_iteration=model.best_iteration_)[:,1]
        sub_train['ans_%s'%str(i)]=train_all
        
        test_proba=model.predict_proba(testX,num_iteration=model.best_iteration_)[:,1]
        sub_test['ans_%s'%str(i)]=test_proba
        
    print("mean f1 in train set：%.5f"%(ft/N))
    print("mean f1 in val set：%.5f"%(fv/N))
    
    return sub_train,sub_test

# ==============ensemble===============

In [64]:
def label_feature(df,N):
    df['label']=0
    for i in range(N):
        df['label']+=df['ans_%s'%str(i)]
    df['label']=df['label']/N
    print(df.shape)
    
    ndf=pd.DataFrame()
    ndf['label']=df['label']
    return ndf

# ========开始跑数据===========

In [65]:
train_data = pd.read_table('../data/train.txt', 
        names= ['prefix','query','title','tag','label'], header= None, encoding='utf-8').astype(str)
val_data = pd.read_table('../data/vali.txt', 
        names = ['prefix','query','title','tag','label'], header = None, encoding='utf-8').astype(str)

test_data = pd.read_table('../data/test.txt',
        names = ['prefix','query','title','tag'],header = None, encoding='utf-8').astype(str)

train_data = train_data[train_data['label'] != '音乐' ]
test_data['label'] = -1

train_data= pd.concat([train_data,val_data])

train_data['label'] = train_data['label'].apply(lambda x: int(x))
test_data['label'] = test_data['label'].apply(lambda x: int(x))

train_label=pd.DataFrame()
train_label['label']=train_data['label']

data=pd.concat([train_data,test_data],axis=0)

# ========建立基本算法模型=========

In [66]:
le=LabelEncoder()

lr=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.08, random_state=1,
                            solver='liblinear', max_iter=100,verbose=1)

lgb= LGBMClassifier(boosting_type='gbdt', num_leaves=60, max_depth=-1, 
                         learning_rate=0.05, n_estimators=2000,objective='binary', 
                         min_split_gain=0,min_child_weight=5, min_child_samples=10, 
                         subsample=0.8, subsample_freq=1,colsample_bytree=0.8, 
                         reg_alpha=3, reg_lambda=5, random_state=1, n_jobs=-1)

clf=LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, 
                         learning_rate=0.05, n_estimators=4000,objective='binary', 
                         min_split_gain=0,min_child_weight=5, min_child_samples=10, 
                         subsample=0.8, subsample_freq=1,colsample_bytree=0.8, 
                         reg_alpha=3, reg_lambda=5, random_state=1, n_jobs=-1)

# ========使用封装函数跑模型=========

In [67]:
data=base_data(data)
traindf,testdf=get_ctr(data)
data=pd.concat([traindf,testdf],axis=0)
print(traindf.shape,testdf.shape,data.shape)

(2049998, 8)
(50000, 8)
(2049998, 92) (50000, 92) (2099998, 92)


In [68]:
testdf=testdf.drop(['label'],axis=1)
for fea in testdf.columns:
    testdf[fea]=testdf[fea].fillna('-1')
    data[fea]=le.fit_transform(data[fea].apply(str))

In [69]:
traindf=data[:train_data.shape[0]]
testdf=data[train_data.shape[0]:]
print(traindf.shape,testdf.shape,data.shape)

(2049998, 92) (50000, 92) (2099998, 92)


In [70]:
testdf=testdf.drop(['label'],axis=1)

In [71]:
#设置好X，y，testX以备后面训练lr，lgb使用
y=traindf.pop('label').values
X=traindf.as_matrix()
testX=testdf.as_matrix()

# ============lightgbm==============

In [72]:
lgb_train,lgb_test=lgb_fit(lgb,X,y,testX,N=10)

Fold=0
Training until validation scores don't improve for 100 rounds.
[100]	train's binary_logloss: 0.325074	valid's binary_logloss: 0.325449
[200]	train's binary_logloss: 0.32244	valid's binary_logloss: 0.323561
Early stopping, best iteration is:
[195]	train's binary_logloss: 0.322479	valid's binary_logloss: 0.323549
best logloss on train_set: 0.32247896204925536
best logloss on test_set: 0.3235487350534136
f1_score on train_set:0.80072
f1_score on test_set:0.79922
Fold=1
Training until validation scores don't improve for 100 rounds.
[100]	train's binary_logloss: 0.325192	valid's binary_logloss: 0.324718
[200]	train's binary_logloss: 0.322565	valid's binary_logloss: 0.322781
[300]	train's binary_logloss: 0.321925	valid's binary_logloss: 0.322984
Early stopping, best iteration is:
[213]	train's binary_logloss: 0.322439	valid's binary_logloss: 0.32275
best logloss on train_set: 0.32243913607099917
best logloss on test_set: 0.3227503133725602
f1_score on train_set:0.80063
f1_score on tes

In [73]:
lgb_train=label_feature(lgb_train,N=10)
print(lgb_train.shape)

lgb_test=label_feature(lgb_test,N=10)
print(lgb_test.shape)

(2049998, 11)
(2049998, 1)
(50000, 11)
(50000, 1)


In [ ]:
lgb_train.to_csv('../data/ensemble/lgb_train1.csv')
lgb_test.to_csv('../data/ensemble/lgb_test1.csv')

# =============LR============

In [ ]:
mm=MinMaxScaler()
X=mm.fit_transform(X)
testX=mm.transform(testX)

lr_train,lr_test=lr_fit(clf,X,y,testX,N=10)

Fold=0
f1 in train set: 0.7958717151610041
f1 in test set: 0.7810613153145818
Fold=1
f1 in train set: 0.7957665515989193
f1 in test set: 0.7823551351168073
Fold=2


In [ ]:
lr_train=label_feature(lr_train,N=10)
print(lr_train.shape)

In [ ]:
lr_test=label_feature(lr_test,N=10)
print(lr_test.shape)

In [ ]:
lr_train.to_csv('../data/ensemble/lr_train1.csv')
lr_test.to_csv('../data/ensemble/lr_test1.csv')

# =========第二层：读取model feature======

In [ ]:
lgb_train=pd.read_csv('../data/ensemble/lgb_train1.csv')
lgb_test=pd.read_csv('../data/ensemble/lgb_test1.csv')

lr_train=pd.read_csv('../data/ensemble/lr_train1.csv')
lr_test=pd.read_csv('../data/ensemble/lr_test1.csv')

ntrain=pd.concat([lgb_train,lr_train],axis=1)
ntrain['label']=train_label['label']
print(ntrain.shape)
print(ntrain.label.value_counts())

ntest=pd.concat([lgb_test,lr_test],axis=1)
print(ntest.shape)

In [ ]:
#设置第二层的X，y，testX
y=ntrain.pop('label').values
X=ntrain.as_matrix()
testX=ntest.as_matrix()

# ========第二层：用lgb训练==============

In [ ]:
ens_train,ens_test=lgb_fit(clf,X,y,testX,N=10)

In [ ]:
label_proba=label_feature(ens_test,N=10)
print(label_proba.shape)

In [ ]:
label_proba['label']=label_proba['label'].apply(np.where(x>0.37,1,0))

In [ ]:
print(label_proba['label'].value_counts())

In [ ]:
# print( /50000)

In [ ]:
# label_proba['label'].to_csv('../data/BaselineV1_5.csv',header=False,index=False)

# ========第二层：用lr训练==============

In [ ]:
ens_train,ens_test=lr_fit(lr,X,y,testX,N=10)

In [ ]:
label_proba=label_feature(ens_test,N=10)
print(label_proba.shape)

In [ ]:
label_proba['label']=label_proba['label'].apply(np.where(x>0.37,1,0))

In [ ]:
print(label_proba['label'].value_counts())